In [2]:
#!/usr/bin/env python
# coding: utf-8


# -*- coding:utf-8 -*-
import pandas as pd
from pandas import Series, DataFrame
import datetime as dt
import re
from functools import reduce
import operator
import datetime
import dateutil.parser
import os
pd.set_option('display.max_columns',None) ##显示所有列
from sqlalchemy import create_engine
from sqlalchemy.types import NVARCHAR, Float, Integer
import pymysql
from WindPy import w
w.start()
from mapping import view_create
from mapping import chn_view_create
# import sys
# reload(sys)
# sys.setdefaultencoding('utf-8')

def get_rate_from_wind(bond_type):
    """
       :func:从wind获取指定类型的债券利率，并导入sql的数据库
             1.先判断表是否存在，若没有直接创建
             2.查看表的最新更新日期，没有默认按照1990. 从wind中提取数据并直接做插入处理
       :param bond_type: 固定参数，债券类型
    """

    eng_table_name = dict_bond_type[bond_type]
    #创建数据库连接
    engine = create_engine('mysql+pymysql://' + user + ":" + password + "@" + db_ip + ":" + str(port) + "/" + db_name,
                           encoding='utf-8')

    id_sql = "select id,eng_name from dict where data_type=\"bond_rate\" and name like "+"\"%%"+bond_type+"%%\""             + " order by dict.index asc"
    ids = engine.execute(id_sql).fetchall()#执行查询
   
    #将查询结果转化为list形式
    id_list=[]
    eng_name_list = []
    for each in ids:
        id_list.append(each[0])
        eng_name_list.append(each[1])

    today = datetime.datetime.today().date()
    end_time = datetime.datetime.today().date()

    table_name = eng_table_name
    tablecheck_result = engine.execute("show tables like \"" + table_name + "\"").rowcount
    
    #1.先判断表是否存在，若有：查看最新日期，从wind中提取数据并直接做插入处理
    if tablecheck_result != 0:
        time_sql = "select date from "+ table_name +" t order by date DESC limit 1" #获取目标表的最新日期
        if engine.execute(time_sql).rowcount == 1:
            latest_mark_time = engine.execute(time_sql).fetchall()[0][0]
            start_time = latest_mark_time + datetime.timedelta(days=1)#获取表的最新日期（冗余代码）

        else:
            start_time = "1990-01-01"
        
        #先检查当前表的最新日期是否需要更新
        if latest_mark_time < today:
            winddata = w.edb(id_list, start_time, today) #从wind获取edb数据
            df = pd.DataFrame()
            df["date"] = winddata.Times
            df["date"] = pd.to_datetime(df['date'])
            #当wind现有数据的最新日期在已有表的最新日期之后 再做数据插入处理
            if  winddata.Times[-1] >= start_time:
                #参数说明：winddata.Codes 返回获取数据的证券代码列表 winddata.Data返回函数获取的数据
                for id, singledata in zip(winddata.Codes, winddata.Data[0]):
                    df[id] = singledata
                '''
                判断是否当天上市！！！
                '''
                df['是否当天上市'] = is_list(df) 
                insert_db(df,table_name)
            else:
                print("no need to insert!")
        #不需要更新
        else:
            print("Data today has already been updated.")
   #2.先判断表是否存在，若没有直接创建
    elif tablecheck_result == 0:
        start_time = "1990-01-01"
        winddata = w.edb(id_list, start_time, today)
        df = pd.DataFrame()
        df["date"] = winddata.Times
        for id, singledata in zip(winddata.Codes, winddata.Data[0]):
            df[id] = singledata
        '''
        判断是否当天上市！！！
        '''
        df['是否当天上市'] = is_list(df) 
        insert_db(df, table_name) #插入数据，函数见下
    else:
        print("duplicate tables exist")


#ExchRepo的单独处理
def get_rate_from_wind_exchrepo():
    
    eng_table_name = "ExchRepo"
    engine = create_engine('mysql+pymysql://' + user + ":" + password + "@" + db_ip + ":" + str(port) + "/" + db_name,
                           encoding='utf-8')

    id_sql = "SELECT id,eng_name FROM dict WHERE data_type=\"bond_rate\" AND dict.name REGEXP '^[A-RT-Z]'  ORDER BY dict.index ASC"
    ids = engine.execute(id_sql).fetchall()
    id_list=[]
    eng_name_list = []
    for each in ids:
        id_list.append(each[0])
        eng_name_list.append(each[1])

    today = datetime.datetime.today().date()
    end_time = datetime.datetime.today().date()

    table_name = eng_table_name
    tablecheck_result = engine.execute("show tables like \"" + table_name + "\"").rowcount
     #1.先判断表是否存在，若有：查看最新日期，从wind中提取数据并直接做插入处理
    if tablecheck_result != 0:
        time_sql = "select date from "+ table_name +" t order by date DESC limit 1"
        if engine.execute(time_sql).rowcount == 1:
            latest_mark_time = engine.execute(time_sql).fetchall()[0][0]#获取表的最新日期（冗余代码）
            start_time = latest_mark_time + datetime.timedelta(days=1)

        else:
            start_time = "1990-01-01"

        if latest_mark_time < today:
            winddata = w.edb(id_list, start_time, today) #从wind获取edb数据
            df = pd.DataFrame()
            df["date"] = winddata.Times
            df["date"] = pd.to_datetime(df['date'])
            
            #当wind现有数据的最新日期在已有表的最新日期之后 再做数据插入处理
            if  winddata.Times[-1] >= start_time:
                for id, singledata in zip(winddata.Codes, winddata.Data):
                    df[id] = singledata
                '''
                判断是否当天上市！！！
                '''
                df['是否当天上市'] = is_list(df) 
                insert_db(df,table_name)
            else:
                print("no need to insert!")
        else:
            print("Data today has already been updated.")
            
    #2.先判断表是否存在，若没有直接创建
    elif tablecheck_result == 0:
        start_time = "1990-01-01"
        winddata = w.edb(id_list, start_time, today)
        df = pd.DataFrame()
        df["date"] = winddata.Times
        for id, singledata in zip(winddata.Codes, winddata.Data):
            df[id] = singledata
        '''
        判断是否当天上市！！！
        '''
        df['是否当天上市'] = is_list(df)
        insert_db(df, table_name)
    else:
        print("duplicate tables exist")

def is_list(data):
    is_list = []
    for i in range(len(data)):
        if data_df.iloc[i]['日期'] == data_df.iloc[i]['issue_listance']:
            is_list.append(True)
        else:
            is_list.append(False)
    return is_list
    

def insert_db(df, table_name):
    """
       :func:将dataframe以指定tablename导入sql的特定数据库中（数据库需存在，不能创建）
       :param df: 固定参数，写入的df数据
       :param table_name: 固定参数，需写入的sql表
    """
    #在执行to_sql方法时，将映射好列名和指定类型的dict赋值给dtype参数，映射函数如下
    dtypedict = mapping_df_types(df)
    engine = create_engine('mysql+pymysql://'+user+":"+password+"@"+db_ip+":"+str(port)+"/"+db_name, encoding='utf-8')
    try:
        df.to_sql(table_name, con=engine, index=False, dtype=dtypedict, if_exists='append')
        #if_exits=append：若表存在，将数据写到原表的后面
        #index：是否将df的index单独写到一列中
        print(" update successful")
    except:
        print("Error to inert,see the detail")


def mapping_df_types(df):
    """
       :func:将pandas.DataFrame中列名和预指定的类型映射起来
       :param df: 固定参数，写入的df数据
       :return:返回dtypedict
    """
    
    dtypedict = {}
    for i, j in zip(df.columns, df.dtypes):
        # if "object" in str(j):
        #     dtypedict.update({i: NVARCHAR(length=255)})
        if "float" in str(j):
            dtypedict.update({i: Float(precision=2, asdecimal=True)})
        if "int" in str(j):
            dtypedict.update({i: Integer()})
    return dtypedict



ModuleNotFoundError: No module named 'WindPy'

In [ ]:
if __name__ == '__main__':
    """
      数据库的全局变量
      user = 'super'
      password = 'Password123'
      db_ip = "mysql57.rdsm05ltcjxv6y8.rds.bj.baidubce.com"
      db_name = 'guanc'
    """

    #全局变量：数据库设置参数
    user = 'root'
    password = 'Welcome123'
    db_ip = "cdb-3gsotx9q.cd.tencentcdb.com"
    db_name = 'rates'
    port = 10059

    # 数据库创建判断
    conn = pymysql.connect(user=user, password=password, host=db_ip, charset='utf8',port=port)
    cur = conn.cursor()
    try:
        cur.execute("create database if not exists " + db_name + " DEFAULT CHARACTER SET utf8")
    except:
        pass
    cur.close()
    conn.close()

    bond_type = ['国债','地方政府债','国开债','口行债','农发行债','中短期票据',
                 '企业债','商业银行二级资本债','商业银行普通债','同业存单','中资美元债',
                 '拆借','质押式回购','互换',"SHIBOR"]
    special_type=["ExchRepo"]

    dict_bond_type={"质押式回购":"REPO",
                   "同业拆借":"interbanklending",
                   "信用拆借":"DepositInistlending",
                    "拆借":"IBlending",
                   "互换":"irs",
                   "口行债":"EXIM",
                   "农发债":"ADB",
                   "国债":"CGB",
                   "国开债":"SDB",
                    "地方政府债":"LGB",
                   "中短期票据":"MTN",
                   "美元债":"USDbond",
                   "企业债":"CORB",
                    "商业银行二级资本债":"CBSB",
                    "商业银行普通债": "BankFinBond",
                    "同业存单":"CDs",
                    "中资美元债": "USDbond",
                   "地方债":"LGB",
                   "农发行债":"ADB",
                   "商业银行":"CB",
                     "SHIBOR":"SHIBOR"
                   }
    for each in bond_type:
        get_rate_from_wind(each)
        # view_create(dict_bond_type[each])
        # chn_view_create(each) # 函数实现怎样的视图功能？？？
    #专门针对ExchRepo进行处理
    get_rate_from_wind_exchrepo()
    # view_create("ExchRepo")
    # chn_view_create("ExchRepo")
